In [ ]:
'''Import packages.'''

## If running for the first time, may need to install packages via terminal.
## If importing errors, check package location and Python kernel match. If not, packages may need to be installed at location of Python kernel.

# Data manipulation.
import numpy as np
import pandas as pd

# Graphing.
import matplotlib as plt
import seaborn as sns

# File access.
import json
import openpyxl

# Statistics.
from scipy import stats
from scipy.ndimage import gaussian_filter1d

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
'''Key analysis variables.'''

## THIS PANEL MUST BE UPDATED FOR ANALYSIS ##

# Set the threshold value for selecting valid frames. Invalid frames are defined as frames with less than 3 pupil markers with a likelihood value above the threshold.
pThreshold = 0.75

# Videos dimensions are 640 x 516 pixels.
fps = 100 # This is rig/camera-dependent.

# Set the saccade threshold. Frames where the eye velocity/displacement is greater than the threshold are considered saccadic.
saccade_threshold = 30/fps # Units are degrees per frame.


In [ ]:
'''Set up file paths.'''

## THIS PANEL MUST BE UPDATED FOR ANALYSIS ##

# Deeplabcut csv file paths; this script is written to take information from two cameras.
dpl1 = '/Users/eugeneliang/Downloads/OKR/OriginalWildTypeAnimals/C57_male_b3.20.24_headpost7.2.24/10/2025-01-16_10-45-17_1DLC_Resnet50_OKR_FellerOct9shuffle4_snapshot_110.csv'
dpl2 = '/Users/eugeneliang/Downloads/OKR/OriginalWildTypeAnimals/C57_male_b3.20.24_headpost7.2.24/10/2025-01-16_10-45-17_2DLC_Resnet50_OKR_FellerOct9shuffle4_snapshot_110.csv'

# Visual stimulus json file path.
dplJson = '/Users/eugeneliang/Downloads/OKR/OriginalWildTypeAnimals/C57_male_b3.20.24_headpost7.2.24/10/10.json'

if dpl1 == dpl2:
    raise Exception('DeepLabCut inputs are identical')

# Identify the frame at which stim appears; needs to be externally determined; if determined from Fiji/ImageJ, subtract by 1. See Instructions for explanation.
# Visual stimulus json file records stimulus timing relative to executing the stimulus program; does not intrinsically adjust to when the cameras begin recording.
stim_appears = 539


In [ ]:
'''Define useful functions.'''

# Calculate distance between two coordinate points.
def distance(x0, y0, x1, y1):
    return np.sqrt((x1-x0)**2 + (y1-y0)**2)


In [ ]:
'''Read in DeepLabCut data for IR cameras 1 and 2 as pandas dataframes.'''

## This Jupyter notebook operates under the assumption that IR camera 1 and IR camera 2 have symmetric data (same number of columns and rows/frames).

ir1_dpl = pd.read_csv(dpl1) # Dataframe with Deeplabcut (dpl) data for IR camera 1 (ir1).
ir2_dpl = pd.read_csv(dpl2) # Dataframe with Deeplabcut (dpl) data for IR camera 2 (ir2).

# Exchange auto-generated column labels for numeric headers.
for df in [ir1_dpl, ir2_dpl]:
    df.columns = [i for i in range(len(df.columns))] 


In [ ]:
'''Partition marker likelihood from DeepLabCut data.'''

# Initialize empty dictionary for DeepLabCut likelihood values.
dplLikelihood = {}

# Generate DeepLabCut likelihood dataframe by selecting every third column from original DeepLabCut data; keys represent the two infrared cameras.
dplLikelihood['ir1'] = pd.DataFrame(data=ir1_dpl[ir1_dpl.columns[::3]].iloc[2:, 1:], dtype=float)
dplLikelihood['ir2'] = pd.DataFrame(data=ir2_dpl[ir2_dpl.columns[::3]].iloc[2:, 1:], dtype=float)

# Clean up likelihood dataframes for further analysis.
for key in dplLikelihood:
    dplLikelihood[key].columns = ir1_dpl.iloc[0].unique()[1:] # Set likelihood dataframe column labels to pupil marker names.
    dplLikelihood[key].reset_index(inplace=True)
    dplLikelihood[key].drop(['index'], axis=1, inplace=True)

# Initialize empty dictionaries for pupil and corneal likelihood values.
pupilLikelihood = {}
cornealLikelihood = {}

# Populate dictionary with pupil and corneal likelihood dataframes for each infrared cameras. Keys are 'ir1' and 'ir2'.
for key in dplLikelihood:
    pupilLikelihood[key] = dplLikelihood[key].iloc[:, :8]
    cornealLikelihood[key] = dplLikelihood[key].iloc[:, 8:]

In [37]:
'''Partition marker locations from DeepLabCut data.'''

# Initialize empty dictionary for Deeplabcut coordinates.
dplMarkers = {}

# Generate DeepLabCut coordinates datraframe by dropping every third column from DeepLabCut.
dplMarkers['ir1'] = pd.DataFrame(data=ir1_dpl.drop(ir1_dpl.columns[::3], axis=1))
dplMarkers['ir2'] = pd.DataFrame(data=ir2_dpl.drop(ir2_dpl.columns[::3], axis=1))

# Clean up markers tables.
for key in dplMarkers:
    dplMarkers[key].columns = dplMarkers[key].iloc[0]
    dplMarkers[key] = dplMarkers[key].iloc[2:].astype(float)
    
    dplMarkers[key].reset_index(inplace=True)
    dplMarkers[key].drop(['index'], axis=1, inplace=True)

# Initialize empty dictionary for separate 1D pupil markers.
dplMarkers1D = {}

# Generate 1-D dataframes (separate x, y coordiantes) for pupil markers in IR camera 1 and camera 2.
for key in dplMarkers:
    dplMarkers1D['x' + str(key)[2]] = dplMarkers[key].iloc[:, range(0, len(dplMarkers['ir1'].columns), 2)]
    dplMarkers1D['y' + str(key)[2]] = dplMarkers[key].iloc[:, range(1, len(dplMarkers['ir1'].columns), 2)]

# Initialize empty dictionary for separate 1D pupil coordinates and corneal reflections.
pupilMarkers = {}
cornealMarkers = {}

# Populate dictionary with pupil and corneal coordinates dataframes for each infrared cameras. Keys are 'ir1' and 'ir2'.
for key in dplMarkers1D:
    pupilMarkers[key] = dplMarkers1D[key].iloc[:, :8]
    cornealMarkers[key] = dplMarkers1D[key].iloc[:, 8:]

# Generate corneal origin dataframe by selecting unique horizontal coordinates and corresponding vertical coordinates.
corneal_origin = pd.DataFrame(data=[cornealMarkers['x1']['CR1'], cornealMarkers['y1']['CR3'], cornealMarkers['x2']['CR2'], cornealMarkers['y2']['CR3']],
                          index=['xO1', 'yO1', 'xO2', 'yO2']
                          ).T

In [ ]:
'''Identfiy the pupil center by approximating a circle bounded by the pupil markers via a least squares solution (Coope method).'''

## This method uses the information from all 8 pupil markers and uses the likelihood values from DeepLabCut to determine whether a frame is valid.
## Valid frames are defined as having at least 3 markers above the likelihood threshold (pThreshold), since 3 points are needed to uniquely define a circle.
## The 'radial center' is still calculated for invalid frames, but the frame number and camera are recorded so that these frames can be adjusted for in later calculations.

# Initialize empty radial center dataframe with column labels.
pupil_rad = pd.DataFrame(columns = ['xC1', 'yC1', 'xC2', 'yC2'])

# Intialize dictionaries for storing information about frames with an insufficient number of points to determine pupil center.
invalidFrames = {'ir1': [], 'ir2': []}
invalCount = {'ir1': 0, 'ir2': 0}

# Calculate radial center for each frame and add to radial center dataframe.
for i in range(len(corneal_origin)):

    # Identify markers with a likelihood value above the threshold.
    sig_markers = {}
    for key in ['ir1', 'ir2']:
        likelihood_i = pupilLikelihood[key].iloc[i]
        sig_markers[key] = likelihood_i[likelihood_i>pThreshold].index
        
        # Check if the current frame has at least 3 markers with a likelihood value greater than the pThreshold.
        if len(sig_markers[key]) < 3:
            
            # Record the index (frame number) and increment the count for invalid frames.
            invalidFrames[key].append(i)
            invalCount[key] += 1

    
    # Slice x, y pupil coordinates for IR camera 1, 2 in each frame as a numpy array.
    vectors_i = {}
    for key in pupilMarkers.keys():
        vectors_i[key] = np.array(pupilMarkers[key].iloc[i][sig_markers['ir' + str(key)[1]]])

    # Set up the linear system.
    matrices_i = {}
    for j in range(1,3):
        matrices_i['A' + str(j)] = np.c_[vectors_i['x' + str(j)], vectors_i['y' + str(j)], np.ones(len(sig_markers['ir' + str(j)]))]
        matrices_i['b' + str(j)] = (vectors_i['x' + str(j)] ** 2) + (vectors_i['y' + str(j)] ** 2)
    
    # Solve the linear system.
    X1=np.linalg.pinv(matrices_i['A1'])@matrices_i['b1']
    X2=np.linalg.pinv(matrices_i['A2'])@matrices_i['b2']
    
    # Add radial center coordinates in either camera for ith frame to radial center dataframe.
    pupil_rad.loc[i] = [X1[0]/2, X1[1]/2, X2[0]/2, X2[1]/2]


In [ ]:
'''Calculate camera scale factor and generate dataframe with pupil angular position in each frame.'''

# Set pupil center data to desired dataframe.
pupil_center = pupil_rad

# Generate dataframe where pupil center coordinates are relative to corneal origin; units are in pixels.
pupil_ang = pupil_center - corneal_origin.values
pupil_ang[pupil_ang.columns[1::2]] = -pupil_ang[pupil_ang.columns[1::2]] # Vertical axis is generated inverted; flip so +x/+y is right/upwards.

# Calculate distance between centers of pupils for each frame in each IR camera.
crocam = np.mean([distance(pupil_ang.loc[i, 'xC1'], pupil_ang.loc[i, 'yC1'], pupil_ang.loc[i, 'xC2'], pupil_ang.loc[i, 'yC2']) for i in range(len(corneal_origin))])

# Calculate scale factor; units are degrees per pixel.
scale = 12 / crocam

# Adjust pupil center position by scale factor; units are in degrees.
pupil_ang *= scale


In [40]:
'''Preview pupil angular position dataframe and scale factor.'''

None

# View first five rows of pupil angular position dataframe, the calculated scale factor, and the count of invalid frames in either camera.
pupil_ang.head(), scale, invalCount

(         xC1        yC1       xC2        yC2
 0  12.832765  17.099171  5.826606  17.214605
 1  12.555178  16.830420  5.771046  17.304988
 2  12.541601  16.763466  5.825064  17.414407
 3  12.621899  17.168715  5.860008  17.633862
 4  12.493570  17.150491  5.732553  17.603420,
 np.float64(0.37069326171935013),
 {'ir1': 52, 'ir2': 371})

In [ ]:
'''Partition experimental json file and extract key visual stimulus information.'''

## For orientations: 0 degrees is nasal/right, 90 is superior/up, 180 is temporal/left, 270 is inferior/down.

# Store json file.
exp_data = json.load(open(dplJson))
exp_stim = exp_data['loggedStimuli'][0]

# Store characteristic variables.
stim_name = exp_stim['protocolName']
stim_sf = exp_stim['spatialFrequency']

# Store key timing variables.
start_offset = exp_stim['_stimulusStartLog'][0]
dirtime = exp_stim['_actualStimTime']

# Define stimulus start frame.
stim_start = (((((pd.Series(exp_stim['_stimulusStartLog'])) - start_offset) + exp_stim['_actualPreTime']) * fps) + stim_appears) // 1
stim_end = (((((pd.Series(exp_stim['_stimulusEndLog'])) - start_offset) - exp_stim['_actualTailTime']) * fps) + stim_appears) // 1

# Initialize empty dictionary for stimulus velocity in each orientation.
stim_vel = {}

# Our dome had an asymmetric distortion that we needed to correct for empircally, which is represented by the hard coded factors.
# Visual stimuli are assigned signs (+/-) according to Cartesian coordinates (i.e. up and right are +, down and left are -).
for dir in [0, 180]:
    stim_vel[dir] = exp_stim['speed']*1.090837954*np.cos(np.deg2rad(dir))
for dir in [90,270]:
    stim_vel[dir] = exp_stim['speed']*1.315847955*np.sin(np.deg2rad(dir))

# # If analyzing data with no dome distortion, run the following lines instead:
# for dir in [0, 180]:
#     stim_vel[dir] = exp_stim['speed']*np.cos(np.deg2rad(dir))
# for dir in [90,270]:
#     stim_vel[dir] = exp_stim['speed']*np.sin(np.deg2rad(dir))

# Generate visual stimulus dataframe, which details the visual stimulus orientation, start frame, and end frame.
vis_stim = pd.DataFrame(data=[exp_stim['_orientationLog'], stim_start, stim_end], index=['orientation', 'start', 'end'], dtype=int)

In [19]:
'''Preview visual stimulus timing.'''

vis_stim

,0
orientation,90
start,1146
end,5646


In [ ]:
'''Calculate slow pursuit displacement.'''

# Parameterize key variables.
camera = min(invalCount)
sigma = 2

# Select stimulus orientation, start frame, and stop frame from visual stimulus dataframe.
stim_orient = vis_stim.loc['orientation', 0]
stim_start = vis_stim.loc['start', 0]
stim_end = vis_stim.loc['end', 0]

# Initialize empty dictionary to hold eye position data.
eye_positions = {}

# Select 1D dataframes from during the visual stimuli.
eye_positions['x'] = pupil_ang['xC' + camera[2]].iloc[stim_start:stim_end]
eye_positions['y'] = pupil_ang['yC' + camera[2]].iloc[stim_start:stim_end]

# Generate invalid frame filter from appropriate camera.
invalidFilter = invalidFrames[camera][stim_start:stim_end]

# Replace invalid frames with the last frame with known positoin.
for key in eye_positions:
    for frame in invalidFilter:
        sub_frame = frame - 1

        # Check if the current invalid frame is the first frame during the visual stimulus presentation.
        if frame == start:
            while sub_frame in invalidFrames[camera] and sub_frame != 0: # Soft check to find the last frame with known position.
                sub_frame -= 1
            eye_positions[key][frame] = pupil_ang[key + 'C' + camera[2]][sub_frame]
        else:
            eye_positions[key][frame] = eye_positions[key][sub_frame]

# Apply a Gaussian filter to eye position data.
for key in eye_positions:
    eye_positions[key] = pd.Series(gaussian_filter1d(eye_positions[key].astype(float), sigma))

# Initialize empty dictionary for eye displacement data.
eye_displacements = {}

# Calculate displacements; eye displacement dictionary inherits keys from eye position dictionary.
for key in eye_positions:
    eye_displacements[key] = np.insert(np.gradient(eye_positions[key], eye_positions[key].index), 0, 0)

# Calculate total displacement of pupil in each frame and generate a mask for saccadic activity.
total_displacement = np.sqrt((eye_displacements['x'] ** 2) + (eye_displacements['y'] ** 2))
saccade_id = total_displacement > saccade_threshold

# Count the number of non-continuous saccadic events and use the saccade mask to replace saccadic frames with 0 displacement in eye displacement dataframes.
numSaccades = np.sum(np.diff(np.concatenate(([0], saccade_id))) == 1)

for key in eye_displacements:
    eye_displacements[key][saccade_id] = 0 

# Integrate filtered displacement data to return positional data with no saccades.
horz_noSaccade = pd.Series(np.cumsum(eye_displacements['x']))
vert_noSaccade= pd.Series(np.cumsum(eye_displacements['y']))

# Calculate gain values.
gain_x = (horz_noSaccade.iloc[-1])/(stim_vel[stim_orient]*dirtime)
gain_y = (vert_noSaccade.iloc[-1])/(stim_vel[stim_orient]*dirtime)


In [ ]:
'''View x gain, y gain, and the number of saccadic events for current replicate.'''

gain_x, gain_y, numSaccades

(np.float64(0.5249627914340943), np.float64(0.3265035536809035), np.int64(51))

In [ ]:
'''Export pupil angular position dataframe to excel sheet.'''

None

## Uncomment and run the  following lines when above data is finalized. To un-comment rows, highlight desired rows and enter (Cntl + /) or (Cmd + /).

# # Export pupil angular position dataframe to excel sheet in local Downloads folder.
# pupil_ang.to_excel('pupil_ang_test.xlsx')